# Week 3 Assignment — MNIST MLP 실험 (TODO 포함)

이 노트북은 3주차 과제를 **그대로 수행**할 수 있도록 구성된 TODO 버전입니다.

## 과제 목표
- MNIST 분류를 위한 **MLP**를 직접 구현한다.
- **Dropout 유무(A/B)** 비교로 Overfitting/Generalization 감각을 얻는다.
- **Optimizer(Adam vs SGD)** 비교로 수렴/성능 차이를 관찰한다.
- 모델 **파라미터 수**를 코드/손계산으로 확인한다.

> ⚠️ `# TODO:`가 있는 부분은 직접 채우세요.  
> ✅ “정확도 높이기”가 목표가 아니라, **실험 → 관찰 → 해석**이 목표입니다.


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# (선택) 재현성
torch.manual_seed(42)

print("torch:", torch.__version__)


torch: 2.5.1


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


## 1) 데이터 준비 (MNIST)

- `train=True`: 학습용(60,000)
- `train=False`: 테스트용(10,000)

전처리는 3주차 MLP 기준으로 **ToTensor()만** 사용합니다.


In [3]:
# TODO: 필요하면 batch_size를 바꿔 실험해보세요.
batch_size = 64

transform = transforms.ToTensor()

train_dataset = datasets.MNIST(
    root="./data",
    train=True,
    transform=transform,
    download=True,
)

test_dataset = datasets.MNIST(
    root="./data",
    train=False,
    transform=transform,
    download=True,
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Train size:", len(train_dataset))
print("Test size:", len(test_dataset))


Train size: 60000
Test size: 10000


In [4]:
# 배치 모양 확인
images, labels = next(iter(train_loader))
print("images shape:", images.shape)  # (B, 1, 28, 28)
print("labels shape:", labels.shape)  # (B,)


images shape: torch.Size([64, 1, 28, 28])
labels shape: torch.Size([64])


## 2) 모델 구현 (TODO)

아래 두 모델을 구현하세요.

- **Model A**: Dropout 없음 (기본 MLP)
- **Model B**: Dropout(p=0.3) 포함

> 힌트: 입력은 (B, 1, 28, 28)이므로 MLP에 넣으려면 **flatten**이 필요합니다.  
> 예) `x = x.view(x.size(0), -1)` 또는 `nn.Flatten()`


In [5]:
class MNISTMLP_NoDropout(nn.Module):
    def __init__(self, hidden1=256, hidden2=128):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.flatten(x)          # (B, 1, 28, 28) -> (B, 784)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)              # logits (softmax는 CrossEntropyLoss 내부에서 처리)
        return x


In [6]:
class MNISTMLP_Dropout(nn.Module):
    def __init__(self, hidden1=256, hidden2=128, p=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, hidden1),
            nn.ReLU(),
            nn.Dropout(p),
            nn.Linear(hidden1, hidden2),
            nn.ReLU(),
            nn.Linear(hidden2, 10),
        )

    def forward(self, x):
        return self.net(x)


## 3) 학습/평가 함수 (TODO)

- `train_one_epoch`: `model.train()` + (zero_grad → forward → loss → backward → step)
- `evaluate`: `model.eval()` + `@torch.no_grad()` (forward만)

> Loss는 기본적으로 **배치 평균**이라서, epoch 평균 loss를 정확히 구하려면  
> `running_loss += loss.item() * batch_size` 후 `running_loss / total_samples`를 사용하세요.


In [7]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()

    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        batch_size = images.size(0)
        running_loss += loss.item() * batch_size

        _, predicted = outputs.max(dim=1)
        total += batch_size
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


In [8]:
@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()

    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        batch_size = images.size(0)
        running_loss += loss.item() * batch_size

        _, predicted = outputs.max(dim=1)
        total += batch_size
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


## 4) 실험 1 — Dropout 유무 비교 (필수)

아래 셀을 완성해서 **Model A(무 Dropout)** vs **Model B(Dropout p=0.3)** 를 비교하세요.

- Epoch: 3~5
- Optimizer: 우선 Adam 또는 SGD 중 하나 고정
- 기록: 각 모델의 Train/Test Loss/Acc

> Tip: 실험을 깔끔히 하려면 **모델/옵티마이저를 새로 생성**하고 돌리세요.


In [9]:
def run_experiment(model, train_loader, test_loader, optimizer, criterion, device, num_epochs=3):
    history = {"train_loss": [], "train_acc": [], "test_loss": [], "test_acc": []}

    for epoch in range(1, num_epochs + 1):
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
        test_loss, test_acc = evaluate(model, test_loader, criterion, device)

        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["test_loss"].append(test_loss)
        history["test_acc"].append(test_acc)

        print(f"Epoch [{epoch}/{num_epochs}] "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
    return history


In [10]:
# 공통 설정
num_epochs = 3

# Loss 함수(분류)
criterion = nn.CrossEntropyLoss()

# -------------------------
# Model A: Dropout 없음
# -------------------------
model_a = MNISTMLP_NoDropout(hidden1=256, hidden2=128).to(device)

# Optimizer (비교 공정성을 위해 exp1에서는 동일 설정 사용)
optimizer_a = optim.Adam(model_a.parameters(), lr=1e-3)

print("=== Model A (No Dropout) ===")
hist_a = run_experiment(model_a, train_loader, test_loader, optimizer_a, criterion, device, num_epochs=num_epochs)


=== Model A (No Dropout) ===
Epoch [1/3] Train Loss: 0.2884, Train Acc: 0.9161 | Test Loss: 0.1412, Test Acc: 0.9558
Epoch [2/3] Train Loss: 0.1109, Train Acc: 0.9666 | Test Loss: 0.1028, Test Acc: 0.9681
Epoch [3/3] Train Loss: 0.0741, Train Acc: 0.9767 | Test Loss: 0.0732, Test Acc: 0.9772


In [11]:
# -------------------------
# Model B: Dropout(p=0.3)
# -------------------------
model_b = MNISTMLP_Dropout(hidden1=256, hidden2=128, p=0.3).to(device)

# Optimizer는 A와 동일하게 (공정 비교)
optimizer_b = optim.Adam(model_b.parameters(), lr=1e-3)

print("=== Model B (Dropout p=0.3) ===")
hist_b = run_experiment(model_b, train_loader, test_loader, optimizer_b, criterion, device, num_epochs=num_epochs)


=== Model B (Dropout p=0.3) ===
Epoch [1/3] Train Loss: 0.3209, Train Acc: 0.9057 | Test Loss: 0.1368, Test Acc: 0.9582
Epoch [2/3] Train Loss: 0.1397, Train Acc: 0.9575 | Test Loss: 0.1059, Test Acc: 0.9673
Epoch [3/3] Train Loss: 0.1019, Train Acc: 0.9687 | Test Loss: 0.0826, Test Acc: 0.9749


### ✍️ 실험 1 서술 (필수)

아래 질문에 **5~7줄**로 답하세요.

1. Train Accuracy는 A/B 중 어느 쪽이 더 높았나요?  
2. Test Accuracy는 A/B 중 어느 쪽이 더 안정적이었나요?  
3. 이를 **Overfitting / Generalization** 관점에서 어떻게 해석하나요?


(여기에 답을 작성하세요)

## 5) 실험 2 — Optimizer 비교 (필수)

같은 모델 구조에서 Optimizer만 바꿔 비교하세요.

- Optimizer A: Adam (권장 lr=1e-3)
- Optimizer B: SGD (권장 lr=0.1)
- Epoch: 1~3

> Tip: 비교를 위해 **같은 모델 구조**를 사용하세요. (Dropout 없는 A 모델 추천)


In [12]:
# 실험 2 설정
num_epochs_opt = 1  # 1~3으로 조절해도 OK

criterion = nn.CrossEntropyLoss()

# 같은 구조의 새 모델 생성 (Dropout 없는 모델로 비교)
model_opt_adam = MNISTMLP_NoDropout(hidden1=256, hidden2=128).to(device)
model_opt_sgd = MNISTMLP_NoDropout(hidden1=256, hidden2=128).to(device)

# Adam optimizer
optimizer_adam = optim.Adam(model_opt_adam.parameters(), lr=1e-3)

# SGD optimizer
optimizer_sgd = optim.SGD(model_opt_sgd.parameters(), lr=0.1)

print("=== Optimizer: Adam ===")
hist_adam = run_experiment(model_opt_adam, train_loader, test_loader, optimizer_adam, criterion, device, num_epochs=num_epochs_opt)

print("\n=== Optimizer: SGD ===")
hist_sgd = run_experiment(model_opt_sgd, train_loader, test_loader, optimizer_sgd, criterion, device, num_epochs=num_epochs_opt)


=== Optimizer: Adam ===
Epoch [1/1] Train Loss: 0.2786, Train Acc: 0.9192 | Test Loss: 0.1333, Test Acc: 0.9585

=== Optimizer: SGD ===
Epoch [1/1] Train Loss: 0.4891, Train Acc: 0.8633 | Test Loss: 0.2349, Test Acc: 0.9266


### ✍️ 실험 2 서술 (필수)

아래 질문에 **4~6줄**로 답하세요.

1. 1 epoch(또는 초반) 기준으로 loss 감소가 더 빨랐던 쪽은?  
2. accuracy 변화는 어땠나요?  
3. “실전에서 하나만 고르라면?” 본인의 선택 기준은?


(여기에 답을 작성하세요)

## 6) 파라미터 개수 분석 (필수)

### (1) 코드로 세기
- `sum(p.numel() for p in model.parameters())`

### (2) 손으로 계산
- Linear(784, 256): 784×256 + 256  
- Linear(256, 128): 256×128 + 128  
- Linear(128, 10): 128×10 + 10  

> 어떤 레이어가 파라미터를 가장 많이 쓰는지 확인하세요.


In [13]:
# 분석할 모델 선택 (예: model_a 또는 model_b 등)
# 여기서는 Model A를 기본으로 사용합니다.
model_for_params = model_a

total_params = sum(p.numel() for p in model_for_params.parameters())
print("Total parameters:", total_params)

# (선택) 레이어별 파라미터 수 보기
for name, p in model_for_params.named_parameters():
    print(name, list(p.shape), p.numel())


Total parameters: 235146
fc1.weight [256, 784] 200704
fc1.bias [256] 256
fc2.weight [128, 256] 32768
fc2.bias [128] 128
fc3.weight [10, 128] 1280
fc3.bias [10] 10


### ✍️ 파라미터 분석 서술 (필수)

아래 질문에 **4~6줄**로 답하세요.

1. 코드로 센 파라미터 수는 얼마인가요?  
2. 손계산 결과(각 레이어 + 합계)는 어떻게 되나요?  
3. 파라미터 수가 커지면 장점/단점은? (표현력↑ / 과적합↑ / 계산량↑ 등)


(여기에 답을 작성하세요)

## 7) (선택) 자유 실험

아래 중 1개 이상 선택해 간단히 결과를 남기세요.
- Dropout p를 0.1 / 0.5로 바꿔보기
- hidden 크기 64/128/256/512 바꿔보기
- epoch 늘려서 Train/Test 차이 관찰
- 틀린 샘플 5개 시각화

(선택 과제는 짧게만 기록해도 됩니다.)
